# Hyperlink graph related experiments 

Creating Hyperlink graph from mongo crawled data.
Importing graph as required in NetworkX.
Running Link analysis algo such as (pagerank, hits).
Using Gephi for visualization and further analysis like (community detection).

In [2]:
# importing all the modules
from pymongo import MongoClient
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import write_dot
from networkx.drawing.nx_agraph import graphviz_layout
from scipy import *
import community
import louvain
import pygraphviz
from pymongo import MongoClient
from simhash import Simhash
from urlparse import urlparse
import redis
import operator
import pandas as pd

In [3]:
# mongo connnection 
client = MongoClient()
collection_name = client['ayush']['collec']

# redis connection
cache = redis.StrictRedis(host='localhost',port=6379, db=0)
cache1= redis.StrictRedis(host='localhost',port=6379, db=0)

In [4]:
# reading and writing graphs from edgelist method
G=nx.read_edgelist("hyperlink_url_parent.edgelist",create_using=nx.DiGraph())
#nx.draw(G,with_labels=False)
#plt.show()





In [5]:
def set_node_attribute(G,node,attribute,value):
    G.node[node][attribute]=value
    

In [6]:
def set_edge_attribute(G,fro,to,attribute,value):
    
    G.edge[fro][to][attribute]=value

In [7]:
# create simhash of the tags
def generate_simhash(data):
        simhash_string = str(Simhash(data).value)
#         simhash_binary = bin(Simhash(data).value)
#         return simhash_string, simhash_binary
        return simhash_string
print generate_simhash("".join("www.nice.org.uks/"))

11971568802140945701


In [8]:

# create a directed graph in NetworkX using EdgeList DS file
def create_directed_graph(edge_list_filename):
    url_graph = nx.read_edgelist(edge_list_filename,create_using=nx.DiGraph())
    return url_graph
link_graph = create_directed_graph('hyperlink_graph_abbvie_edgelist.edgelist')

In [9]:
# function to write a graph in GML for persistent storage and usable some other tools in future
def store_graph_gml(graph):
    nx.write_gml(graph, "hyperlink_graph_abbvie.gml")
    
store_graph_gml(link_graph)

In [10]:
# function to read GML grpah
def read_graph_gml(gml_graph_filename):
    gml_graph = nx.read_gml(gml_graph_filename)
    return gml_graph

link_graph = read_graph_gml('hyperlink_graph_abbvie.gml')

In [11]:
# store the url and hash values for future lookup (in redis)
def store_url_hash(hash_value,url):
    cache.setnx(hash_value,url)
    

In [12]:
def store_mongo_id(url,url_id):
    cache.setnx(url,url_id)
    

In [13]:
def url_hash_lookup(hash_value):
    return cache.get(hash_value)

url_hash_lookup(290356238619843879)

'www.nice.org.uk/news'

In [14]:
# method to read documents from mongo collection and create an Index file of urls/referer_urls 

def read_referer_url(coll_name):
    document_cursor = coll_name.find()
    #edge_file = open('hyperlink_url_parent.edgelist','w')
    
    for document in document_cursor:
        edge_line = ''
        
        if 'referer_url' in document:
            referer_url = document['referer_url']
            scheme_stripped_referer_url = referer_url.split('://')[1]
            
            if '/' in scheme_stripped_referer_url[-1]:
                scheme_stripped_referer_url = scheme_stripped_referer_url[0:-1]
            
            # get simhash of the referer url
            hash_refer_url = generate_simhash(scheme_stripped_referer_url)
            
            edge_line+=hash_refer_url+' '
            store_url_hash(hash_refer_url,scheme_stripped_referer_url)
            set_node_attribute(G,hash_refer_url,'refer_url',str(scheme_stripped_referer_url))
            
       
            
        if  'url' in document:
            url = document['url']
            scheme_stripped_url = url.split('://')[1]
            
            if '/' in scheme_stripped_url[-1]:
                scheme_stripped_url = scheme_stripped_url[0:-1]
                
            # get simhash of the url
            hash_url = generate_simhash(scheme_stripped_url)
            edge_line+=hash_url+' {}'
            store_url_hash(hash_url,scheme_stripped_url)
            store_mongo_id(document['url'],document['_id'])
            
        #if edge_line!='':
           # edge_file.write(edge_line + "\n")
            
        set_node_attribute(G,hash_url,'mongo_id',document['_id'])
        set_edge_attribute(G,hash_refer_url,hash_url,'link_text',document['link_text'])
        
        
        
       
            
read_referer_url(collection_name)


#G.node['5729852446781314082']['mongo_id']
#G.node['9656718576190625029']['refer_url']
#G.edge['10315457602573960478']['9656718576190625029']['link_text']

In [15]:
# neighbourhood graph
def neighbour_graph(src_node_hash,coll_name):
    H=nx.DiGraph()
    for x in G.predecessors(src_node_hash):
        for y in G.successors(x):
            H.add_edge(x,y)
    for x in G.successors(src_node_hash):
        H.add_edge(src_node_hash,x)
    return H

      
H=neighbour_graph('3893340003815868885',collection_name)
nx.draw(H,with_labels=True)
#plt.show()

        

/usr/lib/pymodules/python2.7/matplotlib/collections.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors_original != 'face':
/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [16]:
#shortest path
def get_shortest_path(G,src,dest):
    path = nx.shortest_path(G,source=src,target=dest)
    print path 
    for i in range (len(path)-1):
        print G.edge[path[i]][path[i+1]]['link_text']
get_shortest_path(G,'10315457602573960478','9656718576190625029')

['10315457602573960478', u'9656718576190625029']
Home


In [17]:
# augmented graph
def aug_graph(src_node_hash,level):
    
    curr_succ_list=[src_node_hash]
    curr_pred_list=[src_node_hash]
    H=nx.DiGraph()
        
    while(level):
        temp_node_list=[]
        for x in curr_succ_list:
            for y in G.successors(x):
                H.add_edge(x,y)
                temp_node_list.append(y)
        curr_succ_list = temp_node_list
        
        temp_node_list=[]
        for x in curr_pred_list:
            for y in G.predecessors(x):
                H.add_edge(y,x)
                temp_node_list.append(y)
        curr_pred_list=temp_node_list
                
        level-=1
    return H
H=aug_graph('10315457602573960478',1)
nx.draw(H,with_labels=False)
#plt.show()
        
        
        
    

In [18]:
def get_all_path(G,src,dest):
    path=nx.all_simple_paths(G,source=src,target=dest)
    for x in path:
        print x
        for i in range (len(x)-1):
            print G.edge[x[i]][x[i+1]]['link_text']
get_all_path(G,'10315457602573960478','9656718576190625029')

['10315457602573960478', '9656718576190625029']
Home


In [19]:
def print_elements(coll_name):
    count =0
    edge_list_filename = open('hyperlink_url_parent.edgelist','a')
    document_cursor = coll_name.find()
    
    for document in document_cursor:
        if document['depth']==2:
            count+=1
            print document['url']
    print count
        
    
    

In [20]:
nx.transitivity(G)

3.465346534653465e-05

In [30]:
i=0
for x in G.edges():
        i=i+1
        print i
        print G.edge[x[0]][x[1]]['link_text']


1
About Evidence Search
2
 NICE Pathways
3
Leave feedback
4
Read our interview with NICE Fellow Elizabeth Robb 
5
Advisers
6
 View achievements
7
 Frequently asked questions
8
Scholars
9
 Find out more
10
Fellows and Scholars zone
11
adopting new health technologies
12
resource impact reports and templates
13
Benefits of implementing our guidance
14
audit and service planning
15
Find resources
16
Find out more
17
Get in touch
18
Read the guide
19
View uptake data
20
return on investment tools
21
Multiple long-term conditions, older people and social care
22
Falls in older people
23
Parkinson’s disease
24
Ectopic pregnancy and miscarriage
25
Barrett’s oesophagus
26
Delirium
27
Acute kidney injury
28
Gynaecological conditions (see urogenital conditions)
29
Care homes, oral health
30
Tuberculosis
31
Critical illness rehabilitation
32
Obesity
33
Fractured neck of femur (see hip fracture)
34
Acute coronary syndromes, hyperglycaemia
35
Acute upper gastrointestinal bleeding
36
Brain cancers
3

829
View details
830
View details
831
View details
832
View details
833
View details
834
View details
835
View details
836
View details
837
View details
838
View details
839
View details
840
View details
841
View details
842
View details
843
View details
844
View details
845
View details
846
View details
847
About the CGU committee
848
View details
849
View details
850
View details
851
View details
852
View details
853
View details
854
View details
855
View details
856
View details
857
View details
858
View details
859
View details
860
View details
861
View details
862
View details
863
View details
864
View details
865
View details
866
View details
867
View details
868
View details
869
View details
870
View details
871
View details
872
View details
873
View details
874
View details
875
View details
876
View details
877
View details
878
View details
879
View details
880
View details
881
View details
882
View details
883
About the DAC committee
884
View details
885
View details
886
View 